## Transorming ingest folders data and storing it in silver layer for future transformations

####  using run commands to access notebook to run codes to configuring the adlsgen2 and connect

In [0]:
%run "../includes/adlsconfig"

In [0]:
%run "../includes/containersconfig"

In [0]:
from pyspark.sql.types import *
from pyspark.sql.functions import *

In [0]:
qualifying_struct_schema = StructType([
                                    StructField("qualifyId", IntegerType(), False),
                                    StructField("raceId", IntegerType(), True),
                                    StructField("driverId", IntegerType(), True),
                                    StructField("constructorId", IntegerType(), True),
                                    StructField("number", IntegerType(), True),
                                    StructField("position", IntegerType(), True),
                                    StructField("q1", StringType(), True),
                                    StructField("q2", StringType(), True),
                                    StructField("q3", StringType(), True),
])

In [0]:
qualifying_df = spark.read.format('json').option('multiline', True).schema(qualifying_struct_schema).option('header', True).load(f"{raw_folder_path}/qualifying/")

In [0]:
qualifying_df.display()

In [0]:
qualifying_final_df = qualifying_df.withColumn("ingestion_date", current_timestamp())\
.withColumnsRenamed({"qualifyId" : "qualify_id", "raceId" : "race_id", "driverId" : "driver_id", "constructorId" : "constructor_id"})

In [0]:
qualifying_final_df.write.format('parquet').mode('overwrite').option('path', f"{silver_folder_path}/qualifying").save()

In [0]:
spark.read.format('parquet').option('header', True).option('inferSchema', True).load(f"{silver_folder_path}/qualifying").display()